# Import the required packages


In [ ]:
import os

from torch.nn import (
    Flatten, LogSoftmax, MaxPool2d, Module, Tanh
)

from torchvision.datasets import FashionMNIST

from aihwkit.nn import AnalogConv2dMapped, AnalogLinearMapped, AnalogSequential

from aihwkit.simulator.presets.web import (
    WebComposerInferenceRPUConfig
)

from aihwkit.experiments.experiments.inferencing import BasicInferencing
from aihwkit.experiments.runners import InferenceCloudRunner


## Create the model

In [ ]:
def create_analog_lenet5_network() -> Module:

    """Returns a LeNet5 inspired analog model."""

    N_CLASSES = 10
    rpu_config = WebComposerInferenceRPUConfig()
    print('rpu_config: ', rpu_config)
    
    channel = [16, 32, 512, 128]
    model = AnalogSequential(
        AnalogConv2dMapped(in_channels=1, out_channels=channel[0], kernel_size=5, stride=1,
                           rpu_config=rpu_config),
        Tanh(),
        MaxPool2d(kernel_size=2),
        AnalogConv2dMapped(in_channels=channel[0], out_channels=channel[1], kernel_size=5, stride=1,
                           rpu_config=rpu_config),
        Tanh(),
        MaxPool2d(kernel_size=2),
        Tanh(),
        Flatten(),
        AnalogLinearMapped(in_features=channel[2], out_features=channel[3], rpu_config=rpu_config),
        Tanh(),
        AnalogLinearMapped(in_features=channel[3], out_features=N_CLASSES, rpu_config=rpu_config),
        LogSoftmax(dim=1)
    )

    return model


## Create an instance of the BasicInferencing class for an inference experiment.

The created Experiment contains the definition of the operation to be performed, but is not executed automatically.  That is the role of the Runners. 

In addition to DATASET, MODEL, BATCH_SIZE, there are:

* WEIGHT_TEMPLATE_ID: the full path of the pickle file.
* REPEAT: is the number of drift inference experiments to repeat.
* I_TIMES: is the duration of the drift experiment in seconds.


In [ ]:
DATASET = FashionMNIST
MODEL = create_analog_lenet5_network()
BATCH_SIZE = 8
REPEATS = 2
I_TIMES = 86400
TEMPLATE_ID = 'hwa-trained-lenet5-mapped'

# Create a Basic Inferencing Experiment
my_experiment = BasicInferencing(
    dataset=DATASET,
    model = MODEL,
    batch_size = BATCH_SIZE,
    weight_template_id = TEMPLATE_ID,
    inference_repeats = REPEATS,
    inference_time = I_TIMES
)


##  Set up required data for the cloud runner

A Runner is the object that controls the execution of an Experiment, setting up the environment and providing a convenient way of starting it and retrieving its results. In the code snippet below we will prepare to create a cloud runner instance.


In [ ]:
NAME = os.getenv('JNAME')
if not NAME:
    NAME = 'CLI_inference'

# Create analog_info and noise_model_info
analog_info = {
    'output_noise_strength': 0.04,
    'adc': 9,
    'dac': 7
}

noise_model_info = {
    'device_id': 'pcm',
    'programming_noise_scale': 1.1,
    'read_noise_scale': 1.2,
    'drift_scale': 1.3,
    'drift_compensation': True,
    'poly_first_order_coef': 1.965,
    'poly_second_order_coef': -1.1731,
    'poly_constant_coef': 0.2635,
    'drift_mean': 7.7,
    'drift_std': 8.8
}

inference_info = {
    'dataset': DATASET,
    'batch_size': BATCH_SIZE,
    'weight_template_id': TEMPLATE_ID,
    'inference_repeats': REPEATS,
    'inference_time': I_TIMES
}


## Get a cloud runner instance
This assumes you have set up the $HOME/.config/aihwkit/aihwkit.cfg with the composer token.  See [Setting up your account](https://aihwkit.readthedocs.io/en/latest/using_experiments.html#setting-up-your-account) for how to.

In [ ]:
cloud_runner = InferenceCloudRunner()

## Submit an inference job to the cloud runner.

In [ ]:
cloud_experiment = cloud_runner.run(my_experiment, analog_info,
                                    noise_model_info, name=NAME, device='gpu')

## Display the cloud experiment ID.

In [ ]:

print('cloud_experiment: ', cloud_experiment)
